In [1]:
import numpy as np
import matplotlib as plt

# Übung 09 - Stabilität

Betrachten Sie das dargestellte mechanische System.
Bestimmen Sie das Gesamtpotential des Systems und leiten Sie daraus Gleichgewichtslagen her.


![image.png](fig/image.png)



Die Drehfeder am unteren Rand des Balkensystems ist durch eine Blattfeder mit Länge $l$ ausgeführt. Sie besteht aus Federstahl mit $E=205000 MPa$ und einem Flächenträgheitsmoment von $I = 6,66 mm^4$. Bestimmen Sie die zugehörige Drehfedersteifigkeit in Abhängigkeit der Länge der Drehfeder.

In [ ]:
E= 205e3  # Modulus of Elasticity in MPa
I = 6.66  # Moment of Inertia in mm^4

def w(x, c):
    """ 
    Displacement of the beam at position x given a constant c.
    """
    return c[0] + c[1]*x + 1/2*c[2]*x**2 + 1/6*c[3]*x**3

def dw_dx(x, c):
    """ 
    First derivative of the displacement function w with respect to x.
    """
    return c[1] + c[2]*x + 1/2*c[3]*x**2

def d2w_dx2(x, c):
    """ 
    Second derivative of the displacement function w with respect to x.
    """
    return c[2] + c[3]*x

def d3w_dx3(x, c):
    """ 
    Third derivative of the displacement function w with respect to x.
    """
    return c[3] * np.ones_like(x)

#### Boundary Conditions ####

def spring_bc(c, L):
    """
    Boundary conditions residuals for given integration constants c and beam length L:
    1. w(0) = 0
    2. w(L) = 0
    3. d2w_dx2(0) = 0
    4. d2w_dx2(L) = -1/(E*I)
    """
    return np.array([w(0, c), w(L, c), d2w_dx2(0, c), d2w_dx2(L, c) + 1/(E*I)])


def solve_beam(L):
    """
    Solve for integration constants c using linearity of BCs:
    bc(c) = A @ c + d0  -> solve A c = -d0
    """
    d0 = spring_bc(np.zeros(4), L)
    cols = [spring_bc(np.eye(4)[i], L) - d0 for i in range(4)]
    A = np.column_stack(cols)
    b = -d0
    c = np.linalg.solve(A, b)
    return c


def determine_spring_constant(L):
    """
    Determine the effective spring constant k of the beam.
    k = M/theta where M = 1 Nmm and theta = w'(L)
    """
    c = solve_beam(L)
    theta = dw_dx(L, c)
    k = 1 / theta  # since M = 1 Nmm
    return k

In [3]:
determine_spring_constant(200)

np.float64(-20479499.999999996)

### Kritische Lasten

In [ ]:
def compute_total_potential(theta, cf, L, F):
    """
    Compute the total potential energy of the system.
    U = 0.5 * cf * theta^2 - F * f
    where cf is the spring constant, theta is the rotation at the free end,
    F is the applied force, and f is the corresponding displacement.
    """
    U = 0.5 * cf * theta**2 - 2 * F * (1-np.cos(theta)) * L
    return U

def dU_dtheta(theta, cf, L, F):
    """First derivative dU/dtheta."""
    return cf * theta - 2 * F * L * np.sin(theta)

def d2U_dtheta(theta, cf, L, F):
    """Second derivative d2U/dtheta2."""
    return cf - F * L * np.cos(theta)

def critical_load_linearized(cf, L):
    """
    Critical load from linearization about theta=0:
    d2U/dtheta2(0) = cf - F*L -> F_cr = cf / L
    """
    return cf / L

# Plot the load-displacement curve

def plot_load_displacement(cf, L, theta_max=np.pi/4, num_points=100):
    thetas = np.linspace(-theta_max, theta_max, num_points)
    forces = (cf * thetas) / (2 * L * np.sin(thetas))
    
    plt.pyplot.figure()
    plt.pyplot.plot(thetas, forces)
    plt.pyplot.xlabel('Rotation θ (rad)')
    plt.pyplot.ylabel('Force F (N)')
    plt.pyplot.title('Load-Displacement Curve')
    plt.pyplot.grid(True)
    plt.pyplot.show()

Welche kritische Last ergibt sich für das Stabilitätsproblem. Überprüfen Sie ihre Berechnungen an dem Experiment.

In der nächsten Aufgabe wird das System variiert. Es hat nur folgende Gestalt.

![image](fig/image2.png)

In diesem Fall ist die Drehfeder durch zwei lineare Federn, die mittels Kragarmen mit den Balken verbunden sind, realisiert. Bestimmen Sie die Federsteifigkeit und die kritische Last des Systems.

